# Stats Forecast


In [1]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA
from statsforecast.utils import AirPassengersDF

df = AirPassengersDF
display(df)

sf = StatsForecast(
    models=[AutoARIMA(season_length=12)],
    freq="ME",
)
sf.fit(df)
sf.predict(h=12, level=[95])

,unique_id,ds,y
0,1.0,1949-01-31,112.0
1,1.0,1949-02-28,118.0
2,1.0,1949-03-31,132.0
3,1.0,1949-04-30,129.0
4,1.0,1949-05-31,121.0
...,...,...,...
139,1.0,1960-08-31,606.0
140,1.0,1960-09-30,508.0
141,1.0,1960-10-31,461.0
142,1.0,1960-11-30,390.0


,unique_id,ds,AutoARIMA,AutoARIMA-lo-95,AutoARIMA-hi-95
0,1.0,1961-01-31,444.309575,421.279342,467.339808
1,1.0,1961-02-28,418.213738,390.227635,446.199842
2,1.0,1961-03-31,446.243407,412.910032,479.576782
3,1.0,1961-04-30,488.234223,450.621361,525.847084
4,1.0,1961-05-31,499.237066,457.694834,540.779298
5,1.0,1961-06-30,562.236186,517.130961,607.341411
6,1.0,1961-07-31,649.236458,600.822427,697.650489
7,1.0,1961-08-31,633.236374,581.727786,684.744962
8,1.0,1961-09-30,535.236400,480.808289,589.664511
9,1.0,1961-10-31,488.236392,431.037766,545.435017


# Darts


In [2]:
from darts.datasets import WeatherDataset
from darts.models import TFTModel
import numpy as np

series = WeatherDataset().load()
# predicting atmospheric pressure
target = series["p (mbar)"][:100]
# optionally, past observed rainfall (pretending to be unknown beyond index 100)
past_cov = series["rain (mm)"][:100]
# future temperatures (pretending this component is a forecast)
future_cov = series["T (degC)"][:106]
# by default, TFTModel is trained using a `QuantileRegression` making it a probabilistic forecasting model
model = TFTModel(
    input_chunk_length=6,
    output_chunk_length=6,
    n_epochs=5,
)
# future_covariates are mandatory for `TFTModel`
model.fit(target, past_covariates=past_cov, future_covariates=future_cov)
# TFTModel is probabilistic by definition; using `num_samples >> 1` to generate probabilistic forecasts
pred = model.predict(6, num_samples=100)
# shape : (forecast horizon, components, num_samples)
pred.all_values().shape
(6, 1, 100)
# showing the first 3 samples for each timestamp
pred.all_values()[:, :, :3]
np.array(
    [
        [[-0.06414202, -0.7188093, 0.52541292]],
        [[0.02928407, -0.40867163, 1.19650033]],
        [[0.77252372, -0.50859694, 0.360166]],
        [[0.9586113, 1.24147138, -0.01625545]],
        [[1.06863863, 0.2987822, -0.69213369]],
        [[-0.83076568, -0.25780816, -0.28318784]],
    ]
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                              | Type                             | Params | Mode 
------------------------------------------------------------------------------------------------
0  | train_metrics                     | MetricCollection                 | 0      | train
1  | val_metrics                       | MetricCollection                 | 0      | train
2  | input_embeddings                  | _MultiEmbedding                  | 0      | train
3  | static_cova

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

array([[[-0.06414202, -0.7188093 ,  0.52541292]],

       [[ 0.02928407, -0.40867163,  1.19650033]],

       [[ 0.77252372, -0.50859694,  0.360166  ]],

       [[ 0.9586113 ,  1.24147138, -0.01625545]],

       [[ 1.06863863,  0.2987822 , -0.69213369]],

       [[-0.83076568, -0.25780816, -0.28318784]]])